## Fama-French Three-Factor Model Implementation

### Author: Jay Parmar

#### Date: 19th Aug 2021

This notebook demonstrates the implementation of a Fama-French Three-Factor model on an Indian stock. The steps for the same are as follows:

1. Read libraries
2. Define stock name, start and end date
3. Download stock historical data
4. Resample it to monthly data
5. Calculate stock monthly returns
6. Read factors data and perform data conversion
7. Merge stock returns and factors data
8. Calculate stock excess returns
9. Create a regression model
10. Analyze regression output
11. Calculate expected stock returns.

The Fama-French Three-Factor Model is an extension of the Capital Asset Pricing Model(CAPM). It aims to describe stock returns through three factors listed below:
- Market risk taken by an investor
- The outperformance of small-cap companies relative to large-cap companies
- The outperformance of high book-to-market value companies versus low book-to-market value companies

It is defined by the following equation:

$$ E(R_i)\ =\ R_f\ +\ \beta_1\ *\ (R_m\ -\ R_f)\ +\ \beta_2\ *\ (SMB)\ +\ \beta_3\ (HML)\ +\ \epsilon $$

Where,

$ R_i $ = Expected stock returns <br>
$ R_f $ = Risk-free rate <br>
$ \beta $ = Beta (co-efficient) of the each factor <br>
$ (R_m\ -\ R_f) $ = Market risk premium <br>
$ SMB(Small\ Minus\ Big) $ = Historical excess returns of small companies over big companies <br>
$ HML(High\ Minus\ Low) $ = Historical excess returns of value stocks over growth stocks <br>

We start with importing necessary libraries.

In [1]:
# Import required libraries
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

Next, we define a stock for which we want to calculate expected returns. We perform modeling on four years of data starting from 1st Jan 2015.

In [2]:
# Define stock, start date and end date
stock = 'TCS.NS'

start_date = '2015-01-01'
end_date = '2019-12-31'

I use `yfinance` library to download historical stock data. I use `Adj Close` prices for computing returns. I intend to perform analysis on the monthly data. Hence, the following code downloads and resamples the daily data to monthly data.

In [3]:
# Download stock data and resample it to monthly frequency
stock_data = yf.download(stock, start_date, end_date)

resampled_data = stock_data[['Adj Close']].resample('M').last()

resampled_data.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
2015-01-31,1089.272339
2015-02-28,1168.399048
2015-03-31,1120.826904
2015-04-30,1081.219360
2015-05-31,1145.556152


Calculate stock returns (percentage returns) and rename the column for easy understanding.

In [4]:
# Calculate monthly returns and rename the column
stock_returns = resampled_data.rename(columns={'Adj Close': 'asset_returns'}).pct_change().dropna()

stock_returns.head()

,asset_returns
Date,
2015-02-28,0.072642
2015-03-31,-0.040716
2015-04-30,-0.035338
2015-05-31,0.059504
2015-06-30,-0.013652


Add a new column that contains year and month, and covert its type to integer. We will use this newly created column as a key to merge two dataframes in the code going forward.

In [5]:
# Create a new column to store months and convert its type to integer
stock_returns['month'] = stock_returns.index.strftime('%Y%m').astype('int')

stock_returns.head()

,asset_returns,month
Date,,
2015-02-28,0.072642,201502
2015-03-31,-0.040716,201503
2015-04-30,-0.035338,201504
2015-05-31,0.059504,201505
2015-06-30,-0.013652,201506


Next, we read the factors data. I have used factors data provided by [Fama French and Momentum Factors: Data Library for Indian Market](http://faculty.iima.ac.in/~iffm/Indian-Fama-French-Momentum/).

In [6]:
# Read factors data
factors_data = pd.read_csv('FourFactorsData.csv')

factors_data.head()

,Month,SMB %,HML %,WML %,Rm %,Rf %,Rm-Rf %
0,199310,1.743275,2.336015,5.472541,-1.431621,0.539989,-1.961665
1,199311,-9.550885,-0.495425,13.678909,17.356034,0.542141,16.728924
2,199312,-3.223633,8.570654,0.178863,10.726437,0.325955,10.370179
3,199401,5.183124,-6.113533,-3.875920,20.376547,0.599581,19.665489
4,199402,0.825280,-3.145383,10.144933,4.356845,0.650502,3.683473


The factors data is represented in the percent form in the dataframe above. We need to convert it as we have stock monthly returns in the decimal format. The following code does that:

In [7]:
# Convert percent values to decimal values
def convert(x):
    return x/100

factors_data_d = factors_data[['SMB %', 'HML %', 'WML %', 'Rm %', 'Rf %', 'Rm-Rf %']].apply(convert)

Rename columns for factor data to make working with them easier.

In [8]:
# Rename columns and add month column
factors_data_d.columns = ['smb', 'hml', 'wml', 'rm', 'rf', 'mkt_er']
factors_data_d['Month'] = factors_data['Month']

factors_data_d.head()

,smb,hml,wml,rm,rf,mkt_er,Month
0,0.017433,0.023360,0.054725,-0.014316,0.005400,-0.019617,199310
1,-0.095509,-0.004954,0.136789,0.173560,0.005421,0.167289,199311
2,-0.032236,0.085707,0.001789,0.107264,0.003260,0.103702,199312
3,0.051831,-0.061135,-0.038759,0.203765,0.005996,0.196655,199401
4,0.008253,-0.031454,0.101449,0.043568,0.006505,0.036835,199402


Merge factors data and stock returns dataframes on months. We do this to get combine stock returns and factors data for whatever months of stock data we have. We will perform modeling on the merged output.

In [9]:
# Merge two dataframes and create a new one
data = stock_returns.merge(factors_data_d, left_on='month', right_on='Month')

data.head()

,asset_returns,month,smb,hml,wml,rm,rf,mkt_er,Month
0,0.072642,201502,0.008015,-0.027601,-0.001335,0.010451,0.006398,0.004029,201502
1,-0.040716,201503,0.026750,-0.081992,0.050160,-0.028614,0.006719,-0.035108,201503
2,-0.035338,201504,0.013234,0.040447,-0.013623,-0.025985,0.005802,-0.031613,201504
3,0.059504,201505,0.004802,-0.049373,0.033835,0.034098,0.006129,0.027807,201505
4,-0.013652,201506,0.048534,-0.080069,0.017100,-0.010585,0.006559,-0.017038,201506


We can rearrange the Fama-French Three-Factor model equation as below:

$$ E(R_i)\ -\ R_f\ =\ \beta_1\ *\ (R_m\ -\ R_f)\ +\ \beta_2\ *\ (SMB)\ +\ \beta_3\ (HML)\ +\ \epsilon $$

The left side of the equation becomes our target variable which is nothing but stock excess returns. We calculate it below:

In [10]:
# Calculate stock excess returns
data['stock_er'] = data['asset_returns'] - data['rf']

Define independent variables as `X` and the target variable as `y` and perform regression analysis to get co-efficient for each factor.

In [11]:
# Independent variables
X = data[['mkt_er', 'smb', 'hml']]

# Target variable
y = data['stock_er']

# Create a regression model
reg = sm.OLS(y, X).fit()

print(reg.summary())

                                 OLS Regression Results                                
Dep. Variable:               stock_er   R-squared (uncentered):                   0.367
Model:                            OLS   Adj. R-squared (uncentered):              0.333
Method:                 Least Squares   F-statistic:                              10.84
Date:                Thu, 19 Aug 2021   Prob (F-statistic):                    1.02e-05
Time:                        22:13:01   Log-Likelihood:                          89.235
No. Observations:                  59   AIC:                                     -172.5
Df Residuals:                      56   BIC:                                     -166.2
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Following are the co-efficients we get from the above model.

In [12]:
# Print beta of market_excess_returns
beta_mkt = reg.params[0]
print('Beta of market excess returns= ', round(beta_mkt, 5))

# Print beta of SMB
beta_smb = reg.params[1]
print('Beta of SMB= ', round(beta_smb, 5))

# Print beta of HML
beta_hml = reg.params[2]
print('Beta of HML= ', round(beta_hml, 5))

Beta of market excess returns=  1.11173
Beta of SMB=  -0.58831
Beta of HML=  -0.5434


The following code calculates all annualized version of all factors:

In [13]:
# Calculate annualized market excess returns
ann_market_er = ((1 + data.mkt_er).cumprod().iloc[-1]) ** (12/len(data)) - 1

print('Annualized market excess returns:', round(ann_market_er, 5))

# Calculate annualized SMB returns
ann_smb_returns = ((1 + data.smb).cumprod().iloc[-1]) ** (12/len(data)) - 1

print('Annualized SMB returns:', round(ann_smb_returns, 5))

# Calculate annualized HML returns
ann_hml_returns = ((1 + data.hml).cumprod().iloc[-1]) ** (12/len(data)) - 1

print('Annualized HML returns:', round(ann_hml_returns, 5))

Annualized market excess returns: 0.00796
Annualized SMB returns: -0.00913
Annualized HML returns: -0.12919


We also compute annualized risk free rate. We will require it to compute stock expected returns:

In [14]:
# Calculate annualized risk free rate
ann_rfr = ((1 + data.rf).cumprod().iloc[-1]) ** (12/len(data)) - 1

print('Annualized risk free rate:', round(ann_rfr, 5))

Annualized risk free rate: 0.06411


Finally, we calculate the expected annualized returns of the stock using the following equation:

$$ E(R_i)\ =\ R_f\ +\ \beta_1\ *\ (R_m\ -\ R_f)\ +\ \beta_2\ *\ (SMB)\ +\ \beta_3\ (HML)\ +\ \epsilon $$

We will ignore the error term for this computation.

In [15]:
expected_returns = ann_rfr + beta_mkt * ann_market_er + beta_smb * ann_smb_returns + beta_hml * ann_hml_returns

print('Expected annualized returns of TCS: %.3f' % (expected_returns * 100))

Expected annualized returns of TCS: 14.853


We can see that expected returns for TCS for 1 year turned out to be 14.853% using the Fama French three factor model.